---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    import matplotlib.pyplot as plt
    '''
    for i in P1_Graphs:
        degrees = i.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(j)/float(nx.number_of_nodes(i)) for j in degree_values]
        plt.bar(degree_values, histogram)
        plt.xlabel('Degree')
        plt.ylabel('Fraction of Nodes')
        plt.show()
    '''
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
              
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [52]:
def salary_predictions():
    
    manage_dict = nx.get_node_attributes(G, 'ManagementSalary')
    depart_dict = nx.get_node_attributes(G, 'Department')
    degree_dict = G.degree()
    degcent_dict = nx.degree_centrality(G)
    closecent_dict = nx.closeness_centrality(G)
    betcen_dict = nx.betweenness_centrality(G, normalized=True, endpoints=False)
    
    aver_neigh_deg = nx.average_neighbor_degree(G)
    k_near_neigh = nx.k_nearest_neighbors(G)
    eigen_cent = nx.eigenvector_centrality(G)
    curr_flow = nx.current_flow_betweenness_centrality(G)

    df1 = pd.DataFrame(list(manage_dict.values()), columns=['ManagementSalary'], index=list(manage_dict.keys()))
    df2 = pd.DataFrame(list(depart_dict.values()), columns=['Department'], index=list(depart_dict.keys())).fillna(0)
    df3 = pd.DataFrame(list(degree_dict.values()), columns=['Degree'], index=list(degree_dict.keys())).fillna(0)
    df4 = pd.DataFrame(list(degcent_dict.values()), columns=['DegreeCentrality'], index=list(degcent_dict.keys())).fillna(0)
    df5 = pd.DataFrame(list(closecent_dict.values()), columns=['ClosebessCentrality'], index=list(closecent_dict.keys())).fillna(0)
    df6 = pd.DataFrame(list(betcen_dict.values()), columns=['BetweennessCentrality'], index=list(betcen_dict.keys())).fillna(0)
    
    df7 = pd.DataFrame(list(aver_neigh_deg.values()), columns=['AverageNearestNeighbor'], index=list(aver_neigh_deg.keys())).fillna(0)
    df8 = pd.DataFrame(list(k_near_neigh.values()), columns=['KNN'], index=list(k_near_neigh.keys())).fillna(0)
    df9 = pd.DataFrame(list(eigen_cent.values()), columns=['EigenvectorCentrality'], index=list(eigen_cent.keys())).fillna(0)
    df10 = pd.DataFrame(list(curr_flow.values()), columns=['CurrentFlowBetweennessCentrality'], index=list(curr_flow.keys())).fillna(0)
    
    df = df1.merge(df2, right_index=True, left_index=True).merge(df3, right_index=True, left_index=True).merge(df4, right_index=True, left_index=True).merge(df5, right_index=True, left_index=True).merge(df6, right_index=True, left_index=True).merge(df7, right_index=True, left_index=True).merge(df8, right_index=True, left_index=True).merge(df9, right_index=True, left_index=True).merge(df10, right_index=True, left_index=True)
    
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    
    df = pd.concat([df, pd.get_dummies(df['Department'], prefix="Department")], axis=1)
    df.drop(['Department'], axis=1, inplace=True)
    
    train = df[(df['ManagementSalary'] == 0.0) | (df['ManagementSalary'] == 1.0)]
    test = df[(df['ManagementSalary'] != 0.0) & (df['ManagementSalary'] != 1.0)]
    
    X_train = train.drop(['ManagementSalary'], axis=1)
    y_train = train['ManagementSalary']
    X_train_norm = preprocessing.normalize(X_train)
    
    X_test = test.drop(['ManagementSalary'], axis=1)
    X_test_norm = preprocessing.normalize(X_test)
    
    clf = GradientBoostingClassifier(random_state=1234, '''learning_rate=0.05''').fit(X_train_norm, y_train)
    scores = list(clf.predict(X_test_norm))
    labels = list(X_test.index)

    
    df2 = pd.DataFrame(scores, index=labels)
    
    return df2

salary_predictions()

,0
1,0.0
2,1.0
5,1.0
8,0.0
14,0.0
18,0.0
27,0.0
30,0.0
31,0.0
34,0.0


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here